# Linear Algebra for AI/ML - Part 5: SVD, Decompositions & Advanced Topics

This final notebook covers SVD, matrix decompositions, and advanced concepts.

**Prerequisites:** Complete Parts 1-4 first.

In [1]:
"""
Setup: Import Required Libraries
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from PIL import Image

np.set_printoptions(precision=4, suppress=True)
print(f"NumPy version: {np.__version__}")

NumPy version: 2.2.6


## 27. Singular Value Decomposition (SVD)

**Every matrix** A (m×n) can be factored as:

**A = UΣVᵀ**

Where:
- **U** (m×m): Left singular vectors (orthogonal)
- **Σ** (m×n): Diagonal matrix of singular values (σ₁ ≥ σ₂ ≥ ... ≥ 0)
- **V** (n×n): Right singular vectors (orthogonal)

**ML Applications:** PCA, image compression, recommender systems, NLP (LSA), matrix approximation.

In [2]:
"""
Singular Value Decomposition (SVD)

Formula: A = UΣVᵀ

Components:
- U: m×m orthogonal matrix (left singular vectors)
  - Columns are eigenvectors of AAᵀ
  - Output directions/principal directions

- Σ: m×n diagonal matrix (singular values)
  - σᵢ ≥ 0 (always non-negative)
  - Ordered: σ₁ ≥ σ₂ ≥ ... ≥ 0
  - Strength of each direction

- V: n×n orthogonal matrix (right singular vectors)
  - Columns are eigenvectors of AᵀA
  - Input directions/feature directions

Key Facts:
1. ALWAYS exists (unlike eigendecomposition)
2. Works for ANY matrix (rectangular, singular, etc.)
3. Generalizes eigendecomposition
4. rank(A) = number of non-zero singular values
5. Best low-rank approximation

ML Applications:
- PCA (equivalent to SVD on centered data)
- Image compression
- Recommender systems (matrix factorization)
- Latent Semantic Analysis (NLP)
- Pseudo-inverse computation
"""

print("="*60)
print("Singular Value Decomposition (SVD)")
print("="*60)

# Example matrix
A = np.array([
    [3, 2, 2],
    [2, 3, -2]
])

print(f"\nMatrix A (2×3):")
print(A)
print(f"Shape: {A.shape}")

# Compute SVD
U, s, Vt = np.linalg.svd(A, full_matrices=True)

print(f"\n" + "-"*60)
print("SVD Components: A = UΣVᵀ")
print("-"*60)

print(f"\nU (left singular vectors, {U.shape}):")
print(U)

print(f"\nSingular values σ ({len(s)}):")
print(s)

print(f"\nVᵀ (right singular vectors transposed, {Vt.shape}):")
print(Vt)

# Reconstruct Σ matrix
Sigma = np.zeros(A.shape)
np.fill_diagonal(Sigma, s)

print(f"\nΣ matrix ({Sigma.shape}):")
print(Sigma)

# Verify reconstruction
A_reconstructed = U @ Sigma @ Vt

print(f"\nReconstructed A = UΣVᵀ:")
print(A_reconstructed)

print(f"\nOriginal A:")
print(A)

print(f"\nMatch? {np.allclose(A_reconstructed, A)} ✓")

# Verify orthogonality
print("\n" + "-"*60)
print("Verifying Orthogonality")
print("-"*60)

UTU = U.T @ U
VTV = Vt.T @ Vt

print(f"\nUᵀU (should be I):")
print(UTU)
print(f"Is identity? {np.allclose(UTU, np.eye(U.shape[0]))} ✓")

print(f"\nVᵀV (should be I):")
print(VTV)
print(f"Is identity? {np.allclose(VTV, np.eye(Vt.shape[0]))} ✓")

# Rank from singular values
print("\n" + "-"*60)
print("Matrix Rank from SVD")
print("-"*60)

rank_svd = np.sum(s > 1e-10)  # Count non-zero singular values
rank_numpy = np.linalg.matrix_rank(A)

print(f"\nSingular values: {s}")
print(f"Non-zero singular values: {rank_svd}")
print(f"Rank from numpy: {rank_numpy}")
print(f"Match? {rank_svd == rank_numpy} ✓")

# Low-Rank Approximation
print("\n" + "="*60)
print("Low-Rank Approximation")
print("="*60)

# Keep only top k singular values
k = 1  # Rank-1 approximation

print(f"\nApproximating with top {k} singular value(s)")

# Zero out smaller singular values
Sigma_k = Sigma.copy()
Sigma_k[:, k:] = 0

A_k = U @ Sigma_k @ Vt

print(f"\nRank-{k} approximation:")
print(A_k)

print(f"\nOriginal A:")
print(A)

# Approximation error (Frobenius norm)
error = np.linalg.norm(A - A_k, 'fro')
original_norm = np.linalg.norm(A, 'fro')
relative_error = error / original_norm

print(f"\nApproximation Error:")
print(f"Frobenius norm: {error:.4f}")
print(f"Relative error: {relative_error*100:.2f}%")

# Connection to Eigendecomposition
print("\n" + "="*60)
print("SVD vs Eigendecomposition")
print("="*60)

# For symmetric matrix, SVD = eigendecomposition
S = np.array([
    [4, 2],
    [2, 3]
])

print(f"\nSymmetric matrix S:")
print(S)

# SVD
U_s, s_s, Vt_s = np.linalg.svd(S)

# Eigendecomposition
eigenvalues, eigenvectors = np.linalg.eigh(S)
# Sort descending
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

print(f"\nSingular values: {s_s}")
print(f"Eigenvalues: {eigenvalues}")
print(f"\nFor symmetric matrices:")
print(f"→ Singular values = |eigenvalues|")
print(f"→ U = V = eigenvectors (up to sign)")
print(f"Match? {np.allclose(s_s, np.abs(eigenvalues))} ✓")

# ML Application: Image Compression
print("\n" + "="*60)
print("ML Application: Image Compression with SVD")
print("="*60)

# Create simple synthetic "image" (8x8 grayscale)
np.random.seed(42)
image = np.random.randint(0, 256, (8, 8)).astype(float)

print(f"\nOriginal 'image' (8×8):")
print(image.astype(int))

# SVD
U_img, s_img, Vt_img = np.linalg.svd(image, full_matrices=False)

print(f"\nSingular values:")
print(s_img)

# Compress with different ranks
ranks_to_try = [1, 2, 4, 8]

print(f"\nCompression with different ranks:")
for k in ranks_to_try:
    if k > len(s_img):
        break
    
    # Reconstruct with top k components
    S_k = np.diag(s_img[:k])
    image_k = U_img[:, :k] @ S_k @ Vt_img[:k, :]
    
    # Error
    error = np.linalg.norm(image - image_k, 'fro')
    
    # Compression ratio
    original_size = image.shape[0] * image.shape[1]
    compressed_size = k * (image.shape[0] + image.shape[1] + 1)
    compression_ratio = original_size / compressed_size
    
    # Variance explained
    variance_explained = np.sum(s_img[:k]**2) / np.sum(s_img**2)
    
    print(f"\nRank-{k}:")
    print(f"  Error (Frobenius): {error:.2f}")
    print(f"  Variance explained: {variance_explained*100:.2f}%")
    print(f"  Compression ratio: {compression_ratio:.2f}x")

print(f"\n→ SVD finds best low-rank approximation")
print(f"→ Trade-off: quality vs. compression")
print(f"→ Used in JPEG, video compression, etc.")

# ML Application: PCA via SVD
print("\n" + "="*60)
print("ML Application: PCA via SVD")
print("="*60)

# Generate data
np.random.seed(42)
n_samples = 100
X = np.random.randn(n_samples, 3)  # 3 features

# Add correlation
X[:, 1] = X[:, 0] + 0.5 * np.random.randn(n_samples)
X[:, 2] = 0.3 * X[:, 0] + 0.3 * X[:, 1] + 0.1 * np.random.randn(n_samples)

print(f"\nData shape: {X.shape}")

# Center the data
X_centered = X - X.mean(axis=0)

# SVD (PCA via SVD)
U_pca, s_pca, Vt_pca = np.linalg.svd(X_centered, full_matrices=False)

# Principal components = Vᵀ (right singular vectors)
principal_components = Vt_pca.T

print(f"\nPrincipal Components (columns of V):")
print(principal_components)

# Variance explained
# Variance = (singular values)² / (n-1)
explained_variance = (s_pca ** 2) / (n_samples - 1)
total_variance = explained_variance.sum()
explained_variance_ratio = explained_variance / total_variance

print(f"\nVariance explained by each component:")
for i, (var, ratio) in enumerate(zip(explained_variance, explained_variance_ratio)):
    print(f"  PC{i+1}: {var:.4f} ({ratio*100:.2f}%)")

# Transform data
X_pca = X_centered @ principal_components

print(f"\nTransformed data shape: {X_pca.shape}")

# Alternatively: X_pca = U @ Σ
Sigma_pca = np.diag(s_pca)
X_pca_alt = U_pca @ Sigma_pca

print(f"\nVerify: X @ V = U @ Σ")
print(f"Match? {np.allclose(X_pca, X_pca_alt)} ✓")

print(f"\n→ SVD is the standard way to compute PCA")
print(f"→ More numerically stable than eigendecomposition")
print(f"→ Works even when XᵀX is singular")

Singular Value Decomposition (SVD)

Matrix A (2×3):
[[ 3  2  2]
 [ 2  3 -2]]
Shape: (2, 3)

------------------------------------------------------------
SVD Components: A = UΣVᵀ
------------------------------------------------------------

U (left singular vectors, (2, 2)):
[[-0.7071 -0.7071]
 [-0.7071  0.7071]]

Singular values σ (2):
[5. 3.]

Vᵀ (right singular vectors transposed, (3, 3)):
[[-0.7071 -0.7071 -0.    ]
 [-0.2357  0.2357 -0.9428]
 [-0.6667  0.6667  0.3333]]

Σ matrix ((2, 3)):
[[5. 0. 0.]
 [0. 3. 0.]]

Reconstructed A = UΣVᵀ:
[[ 3.  2.  2.]
 [ 2.  3. -2.]]

Original A:
[[ 3  2  2]
 [ 2  3 -2]]

Match? True ✓

------------------------------------------------------------
Verifying Orthogonality
------------------------------------------------------------

UᵀU (should be I):
[[1. 0.]
 [0. 1.]]
Is identity? True ✓

VᵀV (should be I):
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0. -0.  1.]]
Is identity? True ✓

------------------------------------------------------------
Matrix Rank fro

## 28. Cholesky Decomposition

For symmetric positive definite matrix A:

**A = LLᵀ**

Where L is lower triangular with positive diagonal.

**ML Application:** Sampling from multivariate Gaussian, solving linear systems efficiently.

In [3]:
"""
Cholesky Decomposition

For symmetric positive definite matrix A:
    A = LLᵀ

Where:
- L is lower triangular
- L has positive diagonal entries
- Like "square root" of a matrix

Requirements:
- A must be symmetric: A = Aᵀ
- A must be positive definite: xᵀAx > 0 for all x ≠ 0

Advantages:
- Faster than LU decomposition (half the operations)
- More numerically stable
- Exploits symmetry

ML Applications:
- Sampling from multivariate Gaussian
- Solving (XᵀX)β = Xᵀy efficiently
- Whitening transformations
- Computing determinants

Note: Covariance matrices are always symmetric positive definite!
"""

print("="*60)
print("Cholesky Decomposition")
print("="*60)

# Create symmetric positive definite matrix
# Method: A = BBᵀ for any B
B = np.array([
    [2, 0],
    [1, 3]
])
A = B @ B.T

print(f"\nSymmetric positive definite matrix A:")
print(A)

# Verify properties
print(f"\nVerify symmetric: A = Aᵀ?")
print(f"{np.allclose(A, A.T)} ✓")

# Check positive definite (all eigenvalues > 0)
eigenvalues = np.linalg.eigvalsh(A)
print(f"\nEigenvalues: {eigenvalues}")
print(f"All positive? {np.all(eigenvalues > 0)} ✓")
print(f"→ Matrix is positive definite ✓")

# Compute Cholesky decomposition
L = np.linalg.cholesky(A)

print(f"\nCholesky factor L (lower triangular):")
print(L)

# Verify A = LLᵀ
A_reconstructed = L @ L.T

print(f"\nReconstructed A = LLᵀ:")
print(A_reconstructed)

print(f"\nOriginal A:")
print(A)

print(f"\nMatch? {np.allclose(A_reconstructed, A)} ✓")

# Properties of L
print("\n" + "-"*60)
print("Properties of Cholesky Factor")
print("-"*60)

print(f"\n1. Lower triangular:")
print(L)
print(f"   Upper triangle is zero? {np.allclose(L, np.tril(L))} ✓")

print(f"\n2. Positive diagonal:")
diag_L = np.diag(L)
print(f"   Diagonal: {diag_L}")
print(f"   All positive? {np.all(diag_L > 0)} ✓")

# Determinant
print("\n" + "-"*60)
print("Computing Determinant via Cholesky")
print("-"*60)

# det(A) = det(L)²
# For triangular matrix, det = product of diagonal
det_L = np.prod(np.diag(L))
det_A_cholesky = det_L ** 2
det_A_direct = np.linalg.det(A)

print(f"\ndet(L) = product of diagonal = {det_L:.4f}")
print(f"det(A) = det(L)² = {det_A_cholesky:.4f}")
print(f"Direct det(A) = {det_A_direct:.4f}")
print(f"Match? {np.isclose(det_A_cholesky, det_A_direct)} ✓")

# Solving Linear Systems
print("\n" + "="*60)
print("Solving Ax = b Efficiently")
print("="*60)

b = np.array([7, 11])

print(f"\nSolve Ax = b where:")
print(f"A = {A.tolist()}")
print(f"b = {b}")

# Method 1: Direct (slow)
x_direct = np.linalg.solve(A, b)

# Method 2: Via Cholesky (fast!)
# Ax = b → LLᵀx = b
# Step 1: Solve Ly = b (forward substitution)
y = np.linalg.solve(L, b)
# Step 2: Solve Lᵀx = y (backward substitution)
x_cholesky = np.linalg.solve(L.T, y)

print(f"\nSolution (direct): {x_direct}")
print(f"Solution (Cholesky): {x_cholesky}")
print(f"Match? {np.allclose(x_direct, x_cholesky)} ✓")

# Verify solution
print(f"\nVerification: Ax = {A @ x_cholesky}")
print(f"              b = {b}")
print(f"Correct? {np.allclose(A @ x_cholesky, b)} ✓")

print(f"\n→ Cholesky is ~2× faster than general solve")
print(f"→ Exploits symmetry and positive definiteness")

# ML Application: Sampling from Multivariate Gaussian
print("\n" + "="*60)
print("ML Application: Sampling Multivariate Gaussian")
print("="*60)

# Want to sample from N(μ, Σ)
mean = np.array([2, 3])
cov = np.array([
    [2, 1],
    [1, 1.5]
])

print(f"\nTarget distribution:")
print(f"Mean: {mean}")
print(f"Covariance:")
print(cov)

# Cholesky decomposition of covariance
L_cov = np.linalg.cholesky(cov)

print(f"\nCholesky factor L:")
print(L_cov)

# Sample from N(μ, Σ) using:
# x = μ + L * z, where z ~ N(0, I)

np.random.seed(42)
n_samples = 1000

# Step 1: Sample from standard normal
Z = np.random.randn(n_samples, 2)

# Step 2: Transform: X = μ + Z @ Lᵀ
X = mean + Z @ L_cov.T

# Verify: compute sample mean and covariance
sample_mean = X.mean(axis=0)
sample_cov = np.cov(X.T)

print(f"\nSample statistics ({n_samples} samples):")
print(f"Sample mean: {sample_mean}")
print(f"True mean: {mean}")
print(f"\nSample covariance:")
print(sample_cov)
print(f"\nTrue covariance:")
print(cov)

print(f"\nClose match? {np.allclose(sample_mean, mean, atol=0.1)} ✓")
print(f"             {np.allclose(sample_cov, cov, atol=0.1)} ✓")

print(f"\n→ Cholesky enables efficient sampling")
print(f"→ Used in Monte Carlo, Bayesian inference, GANs")

Cholesky Decomposition

Symmetric positive definite matrix A:
[[ 4  2]
 [ 2 10]]

Verify symmetric: A = Aᵀ?
True ✓

Eigenvalues: [ 3.3944 10.6056]
All positive? True ✓
→ Matrix is positive definite ✓

Cholesky factor L (lower triangular):
[[2. 0.]
 [1. 3.]]

Reconstructed A = LLᵀ:
[[ 4.  2.]
 [ 2. 10.]]

Original A:
[[ 4  2]
 [ 2 10]]

Match? True ✓

------------------------------------------------------------
Properties of Cholesky Factor
------------------------------------------------------------

1. Lower triangular:
[[2. 0.]
 [1. 3.]]
   Upper triangle is zero? True ✓

2. Positive diagonal:
   Diagonal: [2. 3.]
   All positive? True ✓

------------------------------------------------------------
Computing Determinant via Cholesky
------------------------------------------------------------

det(L) = product of diagonal = 6.0000
det(A) = det(L)² = 36.0000
Direct det(A) = 36.0000
Match? True ✓

Solving Ax = b Efficiently

Solve Ax = b where:
A = [[4, 2], [2, 10]]
b = [ 7 11]

Soluti

## 29. Trace & Covariance Matrix

**Trace:** Sum of diagonal elements

**Covariance Matrix:** Measures how features vary together

**ML Application:** PCA, regularization, portfolio optimization.

In [4]:
"""
Trace and Covariance Matrix

TRACE:
- tr(A) = sum of diagonal elements = Σᵢ aᵢᵢ
- Only defined for square matrices

Properties:
- tr(A + B) = tr(A) + tr(B)
- tr(cA) = c·tr(A)
- tr(AB) = tr(BA) (cyclic property)
- tr(A) = sum of eigenvalues
- tr(Aᵀ) = tr(A)

COVARIANCE MATRIX:
- Cov(X) = E[(X - μ)(X - μ)ᵀ]
- Sample: Cov = (1/(n-1)) * XᵀX (centered data)
- Always symmetric
- Always positive semi-definite
- Diagonal = variances of features
- Off-diagonal = covariances between features

ML Applications:
- Trace: total variance, model complexity
- Covariance: PCA, Gaussian processes, portfolio theory
"""

print("="*60)
print("Trace of a Matrix")
print("="*60)

A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

print(f"\nMatrix A:")
print(A)

# Compute trace
trace_manual = A[0,0] + A[1,1] + A[2,2]
trace_numpy = np.trace(A)

print(f"\nTrace = sum of diagonal")
print(f"      = {A[0,0]} + {A[1,1]} + {A[2,2]}")
print(f"      = {trace_manual}")

print(f"\nNumPy: {trace_numpy}")

# Trace properties
print("\n" + "-"*60)
print("Trace Properties")
print("-"*60)

B = np.array([
    [2, 1, 0],
    [0, 3, 2],
    [1, 0, 4]
])

# Property 1: tr(A + B) = tr(A) + tr(B)
trace_sum = np.trace(A + B)
sum_traces = np.trace(A) + np.trace(B)

print(f"\n1. Linearity: tr(A + B) = tr(A) + tr(B)")
print(f"   tr(A + B) = {trace_sum}")
print(f"   tr(A) + tr(B) = {sum_traces}")
print(f"   Equal? {trace_sum == sum_traces} ✓")

# Property 2: tr(AB) = tr(BA)
trace_AB = np.trace(A @ B)
trace_BA = np.trace(B @ A)

print(f"\n2. Cyclic: tr(AB) = tr(BA)")
print(f"   tr(AB) = {trace_AB}")
print(f"   tr(BA) = {trace_BA}")
print(f"   Equal? {trace_AB == trace_BA} ✓")

# Property 3: tr(A) = sum of eigenvalues
eigenvalues = np.linalg.eigvals(A)
sum_eigenvalues = np.sum(eigenvalues)

print(f"\n3. Eigenvalue sum: tr(A) = Σλᵢ")
print(f"   tr(A) = {trace_numpy}")
print(f"   Σλᵢ = {sum_eigenvalues:.4f}")
print(f"   Equal? {np.isclose(trace_numpy, sum_eigenvalues.real)} ✓")

# Covariance Matrix
print("\n" + "="*60)
print("Covariance Matrix")
print("="*60)

# Generate sample data
np.random.seed(42)
n_samples = 100
n_features = 3

X = np.random.randn(n_samples, n_features)
# Add correlations
X[:, 1] = X[:, 0] + 0.5 * np.random.randn(n_samples)
X[:, 2] = -0.8 * X[:, 0] + 0.3 * np.random.randn(n_samples)

print(f"\nData shape: {X.shape}")
print(f"Sample mean: {X.mean(axis=0)}")

# Compute covariance
cov_matrix = np.cov(X.T)  # Note: transpose!

print(f"\nCovariance matrix:")
print(cov_matrix)

# Interpret
print(f"\nInterpretation:")
print(f"Diagonal (variances):")
for i in range(n_features):
    print(f"  Feature {i}: var = {cov_matrix[i, i]:.4f}")

print(f"\nOff-diagonal (covariances):")
print(f"  Features 0 & 1: cov = {cov_matrix[0, 1]:.4f} (positive → move together)")
print(f"  Features 0 & 2: cov = {cov_matrix[0, 2]:.4f} (negative → move opposite)")
print(f"  Features 1 & 2: cov = {cov_matrix[1, 2]:.4f}")

# Properties
print("\n" + "-"*60)
print("Covariance Matrix Properties")
print("-"*60)

# Symmetric
print(f"\n1. Symmetric: Cov = Covᵀ")
print(f"   {np.allclose(cov_matrix, cov_matrix.T)} ✓")

# Positive semi-definite
eigenvalues_cov = np.linalg.eigvalsh(cov_matrix)
print(f"\n2. Positive semi-definite: all eigenvalues ≥ 0")
print(f"   Eigenvalues: {eigenvalues_cov}")
print(f"   All ≥ 0? {np.all(eigenvalues_cov >= -1e-10)} ✓")

# Trace = total variance
trace_cov = np.trace(cov_matrix)
total_var = np.sum(np.var(X, axis=0, ddof=1))

print(f"\n3. Trace = total variance")
print(f"   tr(Cov) = {trace_cov:.4f}")
print(f"   Σ var(xᵢ) = {total_var:.4f}")
print(f"   Equal? {np.isclose(trace_cov, total_var)} ✓")

# ML Application: Trace in Regularization
print("\n" + "="*60)
print("ML Application: Trace in Model Complexity")
print("="*60)

# Weight matrix
W = np.random.randn(5, 5)

print(f"\nWeight matrix W (5×5):")
print(W)

# Trace as measure of model complexity
trace_W = np.trace(W @ W.T)

print(f"\ntr(WWᵀ) = {trace_W:.4f}")
print(f"\nThis equals ||W||²_F (Frobenius norm squared):")
frob_squared = np.linalg.norm(W, 'fro')**2
print(f"||W||²_F = {frob_squared:.4f}")
print(f"Match? {np.isclose(trace_W, frob_squared)} ✓")

print(f"\n→ Trace used in regularization penalties")
print(f"→ Minimizing trace → smaller weights")
print(f"→ Prevents overfitting")

# ML Application: Total Variance in PCA
print("\n" + "="*60)
print("ML Application: Total Variance in PCA")
print("="*60)

# PCA on our correlated data
X_centered = X - X.mean(axis=0)
cov_X = np.cov(X_centered.T)

# Total variance (before PCA)
total_variance_before = np.trace(cov_X)

print(f"\nTotal variance (tr(Cov)): {total_variance_before:.4f}")

# Eigendecomposition
eigenvalues_pca, eigenvectors_pca = np.linalg.eigh(cov_X)
idx = eigenvalues_pca.argsort()[::-1]
eigenvalues_pca = eigenvalues_pca[idx]

print(f"\nEigenvalues (variance per component):")
print(eigenvalues_pca)

# Sum of eigenvalues = trace
sum_eigenvalues = np.sum(eigenvalues_pca)

print(f"\nSum of eigenvalues: {sum_eigenvalues:.4f}")
print(f"Trace of covariance: {total_variance_before:.4f}")
print(f"Match? {np.isclose(sum_eigenvalues, total_variance_before)} ✓")

print(f"\n→ Total variance is preserved in PCA")
print(f"→ Just redistributed among components")
print(f"→ tr(Cov_original) = tr(Cov_transformed)")

Trace of a Matrix

Matrix A:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Trace = sum of diagonal
      = 1 + 5 + 9
      = 15

NumPy: 15

------------------------------------------------------------
Trace Properties
------------------------------------------------------------

1. Linearity: tr(A + B) = tr(A) + tr(B)
   tr(A + B) = 24
   tr(A) + tr(B) = 24
   Equal? True ✓

2. Cyclic: tr(AB) = tr(BA)
   tr(AB) = 76
   tr(BA) = 76
   Equal? True ✓

3. Eigenvalue sum: tr(A) = Σλᵢ
   tr(A) = 15
   Σλᵢ = 15.0000
   Equal? True ✓

Covariance Matrix

Data shape: (100, 3)
Sample mean: [ 0.0918  0.1452 -0.0902]

Covariance matrix:
[[ 0.6803  0.6608 -0.5214]
 [ 0.6608  0.8368 -0.4756]
 [-0.5214 -0.4756  0.5008]]

Interpretation:
Diagonal (variances):
  Feature 0: var = 0.6803
  Feature 1: var = 0.8368
  Feature 2: var = 0.5008

Off-diagonal (covariances):
  Features 0 & 1: cov = 0.6608 (positive → move together)
  Features 0 & 2: cov = -0.5214 (negative → move opposite)
  Features 1 & 2: cov = -0.4756

-------

In [ ]:
"""
Additional Advanced Topics - Quick Reference

These are brief demonstrations of remaining concepts.
Each could be expanded into full sections if needed.
"""

print("="*60)
print("Quick Reference: Additional Topics")
print("="*60)

# 1. Cauchy-Schwarz Inequality
print("\n" + "-"*60)
print("1. Cauchy-Schwarz Inequality")
print("-"*60)

u = np.array([1, 2, 3])
v = np.array([4, 5, 6])

lhs = np.abs(np.dot(u, v))
rhs = np.linalg.norm(u) * np.linalg.norm(v)

print(f"\n|u·v| ≤ ||u|| ||v||")
print(f"{lhs:.4f} ≤ {rhs:.4f}")
print(f"Holds? {lhs <= rhs} ✓")
print("ML: Ensures cosine similarity ∈ [-1, 1]")

# 2. QR Decomposition
print("\n" + "-"*60)
print("2. QR Decomposition")
print("-"*60)

A = np.random.randn(4, 3)
Q, R = np.linalg.qr(A)

print(f"\nA = QR")
print(f"Q shape: {Q.shape} (orthogonal)")
print(f"R shape: {R.shape} (upper triangular)")
print(f"QᵀQ = I? {np.allclose(Q.T @ Q, np.eye(Q.shape[1]))} ✓")
print("ML: Least squares, Gram-Schmidt orthogonalization")

# 3. Pseudo-Inverse
print("\n" + "-"*60)
print("3. Pseudo-Inverse (Moore-Penrose)")
print("-"*60)

A = np.random.randn(5, 3)  # Tall matrix
A_pinv = np.linalg.pinv(A)

print(f"\nA shape: {A.shape}")
print(f"A⁺ shape: {A_pinv.shape}")
print(f"A⁺A = I? {np.allclose(A_pinv @ A, np.eye(3))} ✓")
print("ML: Solving overdetermined systems, linear regression")

# 4. Diagonal Matrix
print("\n" + "-"*60)
print("4. Diagonal Matrix")
print("-"*60)

d = np.array([2, 3, 4])
D = np.diag(d)

print(f"\nDiagonal matrix D:")
print(D)
print(f"\ndet(D) = product of diagonal = {np.prod(d)}")
print(f"D⁻¹ = diag(1/dᵢ)")
print("ML: Scaling transformations, covariance of independent variables")

# 5. Rank-Nullity Theorem
print("\n" + "-"*60)
print("5. Rank-Nullity Theorem")
print("-"*60)

A = np.random.randn(4, 6)
rank = np.linalg.matrix_rank(A)
n = A.shape[1]  # Number of columns
nullity = n - rank

print(f"\nrank(A) + nullity(A) = n")
print(f"{rank} + {nullity} = {n}")
print(f"Verified? {rank + nullity == n} ✓")
print("ML: Understanding model capacity, feature redundancy")

# 6. Gram Matrix
print("\n" + "-"*60)
print("6. Gram Matrix")
print("-"*60)

X = np.random.randn(5, 3)
G = X.T @ X  # Gram matrix

print(f"\nGram matrix G = XᵀX")
print(f"Shape: {G.shape}")
print(f"Symmetric? {np.allclose(G, G.T)} ✓")
print(f"Positive semi-definite? {np.all(np.linalg.eigvalsh(G) >= -1e-10)} ✓")
print("ML: Kernel methods, SVMs, kernel ridge regression")

# 7. Spectral Norm
print("\n" + "-"*60)
print("7. Spectral Norm (Operator Norm)")
print("-"*60)

A = np.random.randn(4, 3)
spectral_norm = np.linalg.norm(A, 2)
_, s, _ = np.linalg.svd(A)
largest_singular_value = s[0]

print(f"\nSpectral norm ||A||₂: {spectral_norm:.4f}")
print(f"Largest singular value: {largest_singular_value:.4f}")
print(f"Equal? {np.isclose(spectral_norm, largest_singular_value)} ✓")
print("ML: Lipschitz constraints, GAN stability, gradient clipping")

# 8. Condition Number
print("\n" + "-"*60)
print("8. Condition Number")
print("-"*60)

A = np.random.randn(5, 5)
cond = np.linalg.cond(A)

print(f"\nCondition number κ(A): {cond:.4f}")
if cond > 1000:
    print("→ Ill-conditioned (sensitive to perturbations)")
else:
    print("→ Well-conditioned")
print("ML: Numerical stability, convergence of optimization")

# Summary
print("\n" + "="*60)
print("Summary: All Major Topics Covered")
print("="*60)

topics = [
    "Fundamentals (Vectors, Matrices, Operations)",
    "Systems & Solutions (Gaussian Elimination, Rank)",
    "Vector Spaces (Basis, Dimension, Span)",
    "Norms & Products (L1, L2, Inner/Outer)",
    "Projections & Orthogonality",
    "Eigenvalues & Eigenvectors",
    "SVD & Decompositions",
    "Covariance & Statistics",
    "Advanced Topics (Pseudo-inverse, QR, etc.)"
]

print("\nTopics covered across all 5 notebooks:")
for i, topic in enumerate(topics, 1):
    print(f"{i}. {topic}")

print("\n→ Complete linear algebra foundation for AI/ML")
print("→ Theory + Code + Applications")
print("→ Ready to understand and implement ML algorithms")

# Linear Algebra for AI/ML - Part 5: Summary

## Core Concepts Summary

| Concept | Key Formula(s) | Properties | ML Application |
|---------|----------------|------------|----------------|
| **SVD (Singular Value Decomposition)** | `A = UΣVᵀ` | Always exists, U & V orthogonal, Σ diagonal with σᵢ ≥ 0 | PCA, image compression, recommender systems, NLP (LSA) |
| **Cholesky Decomposition** | `A = LLᵀ` | L lower triangular with positive diagonal, requires A symmetric positive definite | Efficient linear solves, multivariate Gaussian sampling, optimization |
| **Trace** | `tr(A) = Σaᵢᵢ` | Linear, cyclic (tr(AB)=tr(BA)), equals sum of eigenvalues | Model complexity measure, regularization, total variance in PCA |
| **Covariance Matrix** | `Cov(X) = (XᵀX)/(n-1)` | Symmetric, positive semi-definite, diagonal=variance, off-diagonal=covariance | PCA, feature correlation analysis, portfolio optimization |

## Additional Advanced Topics

| Topic | Description | ML Application |
|-------|-------------|----------------|
| **QR Decomposition** | `A = QR` where Q orthogonal, R upper triangular | Least squares, Gram-Schmidt orthogonalization |
| **Pseudo-Inverse** | `A⁺` (Moore-Penrose) | Solving overdetermined systems, linear regression |
| **Gram Matrix** | `G = XᵀX` | Kernel methods, SVMs, kernel ridge regression |
| **Spectral Norm** | `||A||₂ = largest σ` | Lipschitz constraints, GAN stability, gradient clipping |
| **Condition Number** | `κ(A) = σ_max/σ_min` | Numerical stability, optimization convergence |

## Key Insights

### 1. **SVD (Most Important Decomposition)**
- **Universal**: Works for any matrix (rectangular, singular, etc.)
- **Low-rank approximation**: Best rank-k approximation via Eckart-Young theorem
- **PCA connection**: SVD on centered data = PCA
- **Numerical stability**: Preferred over eigendecomposition for covariance

### 2. **Cholesky for Efficiency**
- **2× faster** than general matrix solves
- **Requires SPD**: Works for covariance matrices, normal equations
- **Sampling**: `x = μ + Lz` where `z ~ N(0,I)` samples from `N(μ, Σ)`

### 3. **Trace & Covariance**
- **Trace = total variance**: `tr(Cov) = Σ variances`
- **Preserved in PCA**: Total variance unchanged, just redistributed
- **Regularization**: `tr(WWᵀ) = ||W||²_F` used in weight decay

### 4. **Covariance Matrix Properties**
- Always **symmetric**: `Cov = Covᵀ`
- Always **positive semi-definite**: eigenvalues ≥ 0
- Diagonal = variances, off-diagonal = covariances

## Practical Applications

| Application | Linear Algebra Concept | Implementation |
|-------------|------------------------|----------------|
| **Image Compression** | SVD low-rank approximation | Keep top-k singular values |
| **Multivariate Gaussian Sampling** | Cholesky decomposition | `x = μ + Lz` where `L = chol(Σ)` |
| **PCA** | SVD of centered data | `U, s, Vt = svd(X_centered)` |
| **Linear Regression** | Normal equations + Cholesky | `(XᵀX)β = Xᵀy` solved via Cholesky |
| **Regularization** | Trace as complexity measure | Add `λ·tr(WWᵀ)` to loss |

## Pro Tips for ML Practitioners

1. **Use SVD for PCA** - more stable than eigendecomposition
2. **Cholesky for (XᵀX)** - when solving normal equations
3. **Check condition numbers** - ill-conditioned matrices cause numerical issues
4. **Trace for regularization** - controls model complexity
5. **Covariance diagonals** - reveal feature scaling needs